In [9]:
import gender_guesser.detector as gender
import pandas as pd
import os
import requests
import urllib.parse #manejo de urls 
from bs4 import BeautifulSoup # Web scrapping TRADUCE LA INFO DE LA PÁGINA PARA DESCARGARLA 
import re

import pandas as pd
import altair as alt 

In [2]:
df= pd.read_excel("Congresistas_Twitter_Peru.xlsx")
df      

,Congresistas,Partido Político,TWITTER
0,"ACATE, Eduardo",Alianza para el Progreso,https://twitter.com/geovanniacate
1,"ACUÑA, Humberto",Alianza para el Progreso,https://twitter.com/HumbertoAcunaP
2,"AGUILAR, Manuel",Acción Popular,x
3,"ALARCON,Edgar",Unión por el Perú,https://twitter.com/EdgarAlarconT
4,"ALENCASTRE, Hirma Norma",Somos Perú,https://twitter.com/NormaAlencastre
...,...,...,...
125,"VERDE, Marco Antonio",Alianza para el Progreso,https://twitter.com/MarcoVerdeH
126,"VIGO, Napoleón Widman",Fuerza Popular,https://twitter.com/napo_vigo
127,"VIVANCO, Miguel",Fuerza Popular,x
128,"YUPANQUI, Mariano",Somos Perú,https://twitter.com/YupanquiMariano


In [3]:
genero=[]
d= gender.Detector()
for i in df["Congresistas"]:
    a=i.split(u" ")
    if len(a)==2:
        f=d.get_gender(a[-1])
        print(a[-1],f)
        genero.append(f)
    elif len(a)==1:
        b=i.split(",")
        f=d.get_gender(b[-1])
        print(b[-1],f)
        genero.append(f)
    else:
        nombre=a[-3]+ u" " + a[-2]+ u" " +a[-1]
        f=d.get_gender(nombre)
        
        print(nombre,f)
        genero.append(f)

Eduardo male
Humberto male
Manuel male
Edgar male
ALENCASTRE, Hirma Norma unknown
Guillermo male
Carlos male
Gilbert male
José male
Yessica unknown
Jesús mostly_male
Walter male
Rita female
Julia female
Wilmer male
Betto unknown
María mostly_female
Lenin unknown
Walter male
Alfredo male
BURGA, Ricardo  unknown
María mostly_female
Rolando male
Irene female
Luis male
Luz mostly_female
Wilmer male
María mostly_female
Posemoscrowte unknown
Hipólito male
Roberto male
Martha female
Lenin unknown
Omar male
Diethell unknown
César male
Julio male
Arlette female
Gino male
Alberto male
Luis male
Kenyon male
ESPINOZA, Samuel Rennan unknown
ESPINOZA, Yeremi Arón unknown
Yessy unknown
Carlos male
Matilde female
Johan male
María mostly_female
Jaqueline female
Paul male
GONZALES,  César unknown
Miguel male
Moises male
Otto male
 unknown
Robledo unknown
Alexander male
Juan male
Nelly female
Leonardo male
Leslye unknown
Mártires unknown
Carolina female
Freddy male
Alexander male
José male
Raúl male
Jim 

In [4]:
df["genero"]=genero

In [5]:
df.reset_index()
df

,Congresistas,Partido Político,TWITTER,genero
0,"ACATE, Eduardo",Alianza para el Progreso,https://twitter.com/geovanniacate,male
1,"ACUÑA, Humberto",Alianza para el Progreso,https://twitter.com/HumbertoAcunaP,male
2,"AGUILAR, Manuel",Acción Popular,x,male
3,"ALARCON,Edgar",Unión por el Perú,https://twitter.com/EdgarAlarconT,male
4,"ALENCASTRE, Hirma Norma",Somos Perú,https://twitter.com/NormaAlencastre,unknown
...,...,...,...,...
125,"VERDE, Marco Antonio",Alianza para el Progreso,https://twitter.com/MarcoVerdeH,unknown
126,"VIGO, Napoleón Widman",Fuerza Popular,https://twitter.com/napo_vigo,unknown
127,"VIVANCO, Miguel",Fuerza Popular,x,male
128,"YUPANQUI, Mariano",Somos Perú,https://twitter.com/YupanquiMariano,male


In [ ]:
lista_generos=[]
counter=0
for i in df["genero"]:
    if i=="male":
        lista_generos.append("Hombre")
    elif i=="female":
        lista_generos.append("Mujer")
    else:
        print(df.iloc[ counter , : ])
        f=input()
        lista_generos.append(f)
    counter+=1

In [7]:
df["genero2"]=lista_generos

In [10]:
response = requests.get("http://www.congreso.gob.pe/pleno/congresistas/")
sopa= BeautifulSoup(response.text, "html.parser")


In [59]:
links=sopa.find_all("tr")

In [80]:
principio="http://www.congreso.gob.pe/pleno/congresistas/"
urls=[]
for i in links[1:]:
    final=i.find_all("a")[0].get("href")
    ulr=principio+final
    urls.append(ulr)

In [104]:
distrito=[]
fecha_inicio=[]
fecha_final=[]
for i in urls: 
    response = requests.get(i)
    sopa= BeautifulSoup(response.text, "html.parser")
    jurisdiccion= sopa.find_all(class_="representa")
    jur=jurisdiccion[0].find(class_="value").get_text()
    distrito.append(jur)
    periodo= sopa.find_all(class_="periodo")
    fechas=periodo[0].find_all(class_="value")
    inicio=fechas[0].get_text()
    final=fechas[1].get_text()
    fecha_inicio.append(inicio)
    fecha_final.append(final)
print(len(fecha_final),len(fecha_inicio),len(distrito))


130 130 130


In [106]:
df["fecha_inicio"]=fecha_inicio
df["fecha_final"]=fecha_final
df["distrito"]=distrito

In [108]:
df["pais"]="Perú"
df["codigo_iso"]="PER"

In [121]:
ARROBA=[]
for i in df["TWITTER"]:
    if i=="x":
        ARROBA.append(i)
    else: 
        a=urllib.parse.urlparse(i)
        a=a[2].split("/")
        arroba="@"+a[1]
        ARROBA.append(arroba)
print(len(ARROBA))
ARROBA

130


['@geovanniacate',
 '@HumbertoAcunaP',
 'x',
 '@EdgarAlarconT',
 '@NormaAlencastre',
 '@guilloaliaga',
 '@AlmeriCarlos',
 '@GilbertAlonzoF',
 '@JoseLuisAncalle',
 '@YessicaApaza6',
 'x',
 '@walterascalde',
 'x',
 '@ayquipa_julia',
 'x',
 '@BettoPorAncash',
 'x',
 '@LeninBazan01',
 '@WCongresista',
 '@AlfredoBenitesF',
 '@ricardoburgaap',
 '@mt_cabrera',
 'x',
 '@ICongresista',
 'x',
 '@LuzCayguaray',
 '@WilmerFrepap',
 '@MariaCespedesc',
 'x',
 '@ChainaHipolito',
 'x',
 '@MarthaChavezC',
 '@leninchecco2020',
 '@OmarChehadeMoya',
 '@ColumbusOpina',
 '@cesarcombina',
 '@FredyCondor4',
 '@arlettecontrers',
 '@CostaGino',
 '@AlbertoBelaunde',
 '@luisrdioses',
 '@durand_kenyon',
 '@rennanespinoza',
 '@ARONESP15730319',
 '@YessyFabinDaz1',
 '@EnriqueFCHA',
 'x',
 '@JohanMoqx',
 'x',
 'x',
 '@PaulGarciaPeru',
 '@QbvXvacxYGGYDZt',
 '@MGonzalesCallao',
 'x',
 '@OttoGuibovich',
 '@RobinsonGupioc',
 '@RobledoGutarra',
 'x',
 '@juanh_champi',
 '@huamani_nelly',
 '@LeonardoIngaAP',
 '@leslye_lazo',

In [122]:
df["arroba"]=ARROBA

In [119]:
a=urllib.parse.urlparse("https://twitter.com/mlizana73")
a[2].split("/")


['', 'mlizana73']

In [136]:
name=[]
counter=0
for i in df["Congresistas"]:
    nombres=i.split(",")
    if nombres[1][0]==u" ":
        nombre=nombres[1][1:]
        apellido=nombres[0]
        completo=nombre+ u" " +apellido[0] +apellido[1:].lower()
        name.append(completo)
    else: 
        nombre=nombres[1]
        apellido =nombres[0]
        completo=nombre+ u" " +apellido[0] +apellido[1:].lower()
        name.append(completo)

In [137]:
df["Congresistas"]=name

In [ ]:
df.to_excel("Peru_legisladores.xlsx")